<a href="https://colab.research.google.com/github/diegotluz/projects-colab/blob/main/Aula_2_6_DTS_PLN_Exerc%C3%ADcios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Exercícios - Aula 2**

Dado o dataset de produtos [1], desenvolva os seguintes pipelines:

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv

Obs.: em todos os pipelines use:
- normalização renovendo valores faltantes
- criem uma nova coluna concatenando as colunas nome e descrição.
- randon_state igual a 42 para permitir a comparação com seus colegas e separe uma amostra de 30% para teste.


In [1]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

**1. ) Treine um modelo de classificação DecisionTreeClassifier do pacote scikit-learn para classificar os produtos em suas categorias, com as seguintes configurações usando a nova coluna (nome + descricao):**

- 1.1 Contagem de termos simples com unigrama e sem stop-words.
- 1.2 Contagem de termos simples com unigrama + brigrama e sem stop-words.
- 1.3 TF-IDF com unigrama e com stop-words.
- 1.4 TF-IDF com unigrama e sem stop-words.
- 1.5 TF-IDF com unigrama e sem stop-words em textos lematizados (Dica: crie uma função para lematizar o texto usando o Spacy).

In [13]:
!pip install scikit-learn pandas nltk spacy >> /dev/null
!python -m spacy download pt_core_news_sm --quiet
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:

import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd


# Carregue o dataset
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

# Limpeza e pré-processamento
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

# Função para lematizar o texto
nlp = spacy.load("pt_core_news_sm")
def lematizar_texto(texto):
    doc = nlp(texto)
    tokens_lematizados = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens_lematizados)

df['texto_lematizado'] = df['texto'].apply(lematizar_texto)

# Divisão da amostra entre treino e teste
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Carregue as stop words do NLTK
stop_words = nltk.corpus.stopwords.words('portuguese')

# Configurações dos modelos
configs = [
    {'vectorizer': CountVectorizer(stop_words=stop_words, ngram_range=(1, 1)), 'text_col': 'texto'},  # 1.1
    {'vectorizer': CountVectorizer(stop_words=stop_words, ngram_range=(1, 2)), 'text_col': 'texto'},  # 1.2
    {'vectorizer': TfidfVectorizer(ngram_range=(1, 1)), 'text_col': 'texto'},  # 1.3
    {'vectorizer': TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1)), 'text_col': 'texto'},  # 1.4
    {'vectorizer': TfidfVectorizer(ngram_range=(1, 1)), 'text_col': 'texto_lematizado'}  # 1.5
]


# Treinamento e avaliação dos modelos
for i, config in enumerate(configs):
    print(f"Treinando modelo {i + 1}...")

    vectorizer = config['vectorizer']
    text_col = config['text_col']

    # Vetorização do dataframe de treino
    vectorizer.fit(df_train[text_col])
    x_train = vectorizer.transform(df_train[text_col])
    y_train = df_train["categoria"]

    # Treinamento do modelo DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(x_train, y_train)

    # Transforma o dataframe de teste em vetor (features)
    x_test = vectorizer.transform(df_test[text_col])

    # Escoragem da classificação na amostra de teste
    y_prediction = model.predict(x_test)

    # Mensuração do resultado pela acurácia
    y_test = df_test["categoria"]
    accuracy = accuracy_score(y_prediction, y_test)
    print(f"Acurácia do modelo {i + 1}: {accuracy}")
    print("-" * 30)  # Separador visual entre os resultados

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Treinando modelo 1...
Acurácia do modelo 1: 0.9611428571428572
------------------------------
Treinando modelo 2...
Acurácia do modelo 2: 0.9565714285714285
------------------------------
Treinando modelo 3...
Acurácia do modelo 3: 0.9531428571428572
------------------------------
Treinando modelo 4...
Acurácia do modelo 4: 0.9565714285714285
------------------------------
Treinando modelo 5...
Acurácia do modelo 5: 0.9314285714285714
------------------------------


In [16]:
df['texto'].lemma_

AttributeError: 'Series' object has no attribute 'lemma_'

**2. ) Treine um modelo de classificação LogisticRegression do pacote scikit-learn para classificar os produtos em suas categorias, com as seguintes configurações:**

- 2.1 Contagem de termos simples com unigrama e com stop-words.
- 2.2 Contagem de termos simples com unigrama + brigrama e sem stop-words.
- 2.3 TF-IDF com unigrama e sem stop-words.
- 2.4 TF-IDF com unigrama e sem stop-words em textos lematizados.

Extra:
- 2.5 Contagem de termos simples (BoW) com unigrama, sem stop-words (combinando Spacy e NLTK) em textos com apenas verbos lematizados.

Dica: crie uma função para lematizar o texto usando o Spacy, não esqueça de usar o POS-Tag quando necessário.

In [ ]:
# reposta

# Carregue o dataset
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

# Limpeza e pré-processamento
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

# Função para lematizar o texto (geral e apenas verbos)
nlp = spacy.load("pt_core_news_sm")
def lematizar_texto(texto, apenas_verbos=False):
    doc = nlp(texto)
    if apenas_verbos:
        tokens_lematizados = [token.lemma_ for token in doc if not token.is_stop and token.pos_ == 'VERB']
    else:
        tokens_lematizados = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens_lematizados)

df['texto_lematizado'] = df['texto'].apply(lematizar_texto)
df['texto_verbos_lematizados'] = df['texto'].apply(lambda x: lematizar_texto(x, apenas_verbos=True))

# Divisão da amostra entre treino e teste
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Carregue as stop words do NLTK
stop_words = nltk.corpus.stopwords.words('portuguese')

# Configurações dos modelos
configs = [
    {'vectorizer': CountVectorizer(ngram_range=(1, 1)), 'text_col': 'texto'},  # 2.1
    {'vectorizer': CountVectorizer(stop_words=stop_words, ngram_range=(1, 2)), 'text_col': 'texto'},  # 2.2
    {'vectorizer': TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1)), 'text_col': 'texto'},  # 2.3
    {'vectorizer': TfidfVectorizer(ngram_range=(1, 1)), 'text_col': 'texto_lematizado'},  # 2.4
    {'vectorizer': CountVectorizer(stop_words=stop_words, ngram_range=(1, 1)), 'text_col': 'texto_verbos_lematizados'}  # 2.5
]

# Treinamento e avaliação dos modelos
for i, config in enumerate(configs):
    print(f"Treinando modelo {i + 1}...")

    vectorizer = config['vectorizer']
    text_col = config['text_col']

    # Vetorização do dataframe de treino
    vectorizer.fit(df_train[text_col])
    x_train = vectorizer.transform(df_train[text_col])
    y_train = df_train["categoria"]

    # Treinamento do modelo LogisticRegression
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(x_train, y_train)

    # Transforma o dataframe de teste em vetor (features)
    x_test = vectorizer.transform(df_test[text_col])

    # Escoragem da classificação na amostra de teste
    y_prediction = model.predict(x_test)

    # Mensuração do resultado pela acurácia
    y_test = df_test["categoria"]
    accuracy = accuracy_score(y_prediction, y_test)
    print(f"Acurácia do modelo {i + 1}: {accuracy}")